In [1]:
import pandas as pd
import os
import numpy as np
from pandas.tseries.offsets import BDay
import win32com.client
import pytz
from datetime import datetime,timedelta,date

In [2]:
SQL_MCID = 0
Path = r'E:\_Projects\Delivery Perforamce\\'

In [3]:
import oracledb
import pandas as pd
import numpy as np

if SQL_MCID == 0:

    # Enable thick mode
    oracledb.init_oracle_client(lib_dir=r"E:\softwares\instantclient-basic-windows.x64-23.5.0.24.07\instantclient_23_5")

    # Connection details
    dsn_tns = oracledb.makedsn('153.84.75.216', 1521, service_name='ebs_ATMPROD')


    # Use a context manager to handle the connection
    with oracledb.connect(user='erp_user', password='erp_pass', dsn=dsn_tns) as conn:
        

        query = f"""
        select
        pha.segment1 as PO,
        pha.vendor_id as VID,
        case
         when pha.vendor_id='10013919' then 'JAB'
         when pha.vendor_id='16783041' then 'ENN'
         else 'USI'
        end as SITE,
        To_char(pll.need_by_date, 'DD-MON-YYYY') As NBD_dt,
        To_char(pll.promised_date, 'DD-MON-YYYY') As PD_dt,
        pll.revision_num,
        pll.quantity as qty,
        pll.LAST_UPDATE_DATE,
        pll.APPROVED_DATE,
        pha.CREATION_DATE
        from
        po_line_locations_archive_all pll,
        po_headers_all pha
        where
        pll.po_header_id = pha.po_header_id
        --and pha.segment1='5001502495'
        and pha.vendor_id in (16783041,10013919,9401862) --16783041 ENNOCONN, 10013919 JABIL, 9401862 USI
        and pha.creation_date>='01-JAN-23'
        and pll.promised_date is not null
        order by segment1, revision_num

        """

        # Fetch data into a pandas DataFrame and append to the main DataFrame
        data_raw = pd.read_sql(query, con=conn)
        
    # Save the combined DataFrame to a CSV file
    data_raw.to_csv(Path+'Output\\ref\\PO_Details.csv', index=False)
else:
    data_raw = pd.read_csv(Path+'Output\\ref\\PO_Details.csv')


C:\Users\HT185078\AppData\Local\Temp\3\ipykernel_29436\345156821.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_raw = pd.read_sql(query, con=conn)


In [4]:
data_last = data_raw.copy()
data_first = data_raw.copy()

data_last.sort_values(by=['PO','SITE','LAST_UPDATE_DATE'])
data_last.drop_duplicates(subset=['PO','SITE'], keep='last',inplace=True)
# data_last.to_csv('E:\_Projects\splr_delivery\snd\data_last.csv',index= False)

data_first.sort_values(by=['PO','SITE','LAST_UPDATE_DATE'])
data_first.drop_duplicates(subset=['PO','SITE'], keep='first',inplace=True)
data_first.to_csv('E:\_Projects\splr_delivery\snd\data_first.csv',index= False)

data_first['PO']= data_first['PO'].astype(str)
data_first['SITE']= data_first['SITE'].astype(str)
first_pocreation = data_first[['SITE','PO','CREATION_DATE']]

first_pocreation['SitePO'] = first_pocreation['SITE'] + first_pocreation['PO']
first_pocreation.to_csv('E:\_Projects\splr_delivery\snd\pocreation.csv',index= False)
 

C:\Users\HT185078\AppData\Local\Temp\3\ipykernel_29436\188796955.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_pocreation['SitePO'] = first_pocreation['SITE'] + first_pocreation['PO']


### User Input

In [5]:
dt = datetime.now()
date_str = dt.strftime("%Y%m%d")
print(date_str)
# data = pd.read_csv(r'./rcv/completion_po_'+date_str+'.csv', encoding='utf-8', low_memory=False)
data = pd.read_csv(r'E:\_Projects\_outputs\completion\Completion PO\completion_po_20251203.csv', encoding='utf-8', low_memory=False)
data = data[['site','item','PO','ship_date','qty','item_type','ship_date_mth','class','prod_grp_wb']]
data= data[data['site'].isin(['BUD','JAB'])]
data= data[data['item_type'].isin(['M'])]
data= data[~data['item'].str.contains('-KIT')]
item = [202511]
data = data[data['ship_date_mth'].isin(item)]
data.to_csv('E:\_Projects\splr_delivery\snd\Shipped_data.csv',index= False)

20251203


In [6]:
data_wrk = data[['site','PO','prod_grp_wb','class','item','item_type','ship_date','ship_date_mth','qty']]
data_wrk['PO']= data_wrk['PO'].astype(str)
data_last['PO']= data_last['PO'].astype(str)
data_wrk.to_csv('E:\_Projects\splr_delivery\snd\data_wrk1.csv',index= False)
data_wrk = data_wrk.merge(data_last[['PO','NBD_DT']], on='PO', how='left')

data_wrk['NBD_DT'] = data_wrk['NBD_DT'].astype('datetime64[ns]')
data_wrk = data_wrk.merge(data_first[['PO','PD_DT']], on='PO', how='left')

data_wrk.rename(columns={'PD_DT' : 'Promise Dt (first)','NBD_DT':'Need By dt'}, inplace=True)
data_wrk = data_wrk.merge(data_last[['PO','PD_DT']], on='PO', how='left')

data_wrk.rename(columns={'PD_DT' : 'Promise Dt (last)'}, inplace=True)

data_wrk = data_wrk.merge(data_first[['PO','APPROVED_DATE']], on='PO', how='left')
data_wrk.rename(columns={'APPROVED_DATE' : 'Approved Dt(first)'}, inplace=True)

data_wrk['Receipt Dt'] =''
data_wrk['Receipt Dt'] = data_wrk['Receipt Dt'].astype('datetime64[ns]')
data_wrk['ship_date'] = data_wrk['ship_date'].astype('datetime64[ns]')
cond = data_wrk['site']=='JAB'
data_wrk['Receipt Dt'] = np.where(cond,data_wrk['ship_date'] + BDay(5),data_wrk['ship_date'] )
data_wrk['SitePO']= data_wrk['site'] + data_wrk['PO']
data_wrk = data_wrk.merge(first_pocreation[['SitePO','CREATION_DATE']], on='SitePO', how='left')
data_wrk.to_csv('E:\_Projects\splr_delivery\snd\data_wrk.csv',index= False)

In [7]:
data_result = data_wrk.copy()
data_result.columns

Index(['site', 'PO', 'prod_grp_wb', 'class', 'item', 'item_type', 'ship_date',
       'ship_date_mth', 'qty', 'Need By dt', 'Promise Dt (first)',
       'Promise Dt (last)', 'Approved Dt(first)', 'Receipt Dt', 'SitePO',
       'CREATION_DATE'],
      dtype='object')

In [8]:
data_result['Delivery vs Orig PD'] =''
data_result['Delivery vs Orig PD'] = data_result['Delivery vs Orig PD'].astype('datetime64[ns]')

data_result['Receipt Dt'] = data_result['Receipt Dt'].astype('datetime64[ns]')
data_result['Promise Dt (first)'] = data_result['Promise Dt (first)'].astype('datetime64[ns]')
data_result['Delivery vs Orig PD'] = data_result['ship_date'] - data_result['Promise Dt (first)']
data_result['Delivery vs Orig PD'] = data_result['Delivery vs Orig PD'] / np.timedelta64(1, 'D')

cond1 = data_result['Delivery vs Orig PD'] <= -5
cond2 = data_result['Delivery vs Orig PD'] <= 1
data_result['Complianc Orig PD'] = np.where(cond1,'On-Time', \
                                           np.where(cond2,'On-Time','Late'))


con1 = data_result['Delivery vs Orig PD'] <= 0
con2 = data_result['Delivery vs Orig PD'] <= 1
con3 = data_result['Delivery vs Orig PD'] <= 5
con4 = data_result['Delivery vs Orig PD'] <= 10
con5 = data_result['Delivery vs Orig PD'] <= 20
con6 = data_result['Delivery vs Orig PD'] <= 30
data_result['Perf Category Orig PD'] = np.where(con1,'Early', \
                                               np.where(con2,'On-Time', \
                                                        np.where(con3,'1) 1-5 Days Late', \
                                                           np.where(con4,'2) 5-10 Days Late', \
                                                               np.where(con5,'3) 10-20 Days Late', \
                                                                    np.where(con6,'4) 20-30 Days Late','5) 30+ Days Late'))))))


data_result['Promise Dt (last)'] = data_result['Promise Dt (last)'].astype('datetime64[ns]')

#data_result['Need By dt']= pd.to_datetime(data['Need By dt'], format='%Y-%m-%d', errors='coerce')


data_result['Need By dt'] = pd.to_datetime(data_result['Need By dt']).dt.date
data_result['Promise Dt (first)'] = pd.to_datetime(data_result['Promise Dt (first)']).dt.date
data_result['Promise Dt (last)'] = pd.to_datetime(data_result['Promise Dt (last)']).dt.date
data_result['Promise Dt (last)'] = pd.to_datetime(data_result['Promise Dt (last)']).dt.date
data_result['Receipt Dt'] = pd.to_datetime(data_result['Receipt Dt']).dt.date
# data_result['PO Creation Dt'] = pd.to_datetime(data_result['PO Creation Dt']).dt.date



data_result['site']=data_result['site'].replace({'BUD':'ENN'})

data_result.rename(columns={'item' : 'PN','ship_date':'SHIPPED DATE','qty':'Qty','CREATION_DATE':'PO Creation Dt','item_type':'PID Type','prod_grp_wb':'Prod Grp','ship_date_mth':'Ship Period','class':'Class'}, inplace=True)

data_result.drop(columns={'SitePO'}, axis=1, inplace=True)    

data_result['IO Booked Dt(First)']=''
data_result['IO Booked Dt(Last)']=''
data_result['Booked Dt']=''

                           
Source = data_result[['site','PO','Prod Grp','Class','PN','PID Type','SHIPPED DATE','Ship Period','Qty','Need By dt','Booked Dt','Promise Dt (first)','Promise Dt (last)','Receipt Dt','PO Creation Dt','Delivery vs Orig PD','Complianc Orig PD','Perf Category Orig PD']]




## OTD Data Process

In [9]:
Product = pd.read_csv(Path+'Input\\class_offerpf_map.csv')



Source['Class'] = Source['Class'].astype('str')
Product['Class'] = Product['Class'].astype('str')

Data = pd.merge(Source, Product[['Class','Offer PF']], on='Class',how='left')


C:\Users\HT185078\AppData\Local\Temp\3\ipykernel_29436\1023442606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Source['Class'] = Source['Class'].astype('str')


#### Update from EUMOCP

In [10]:
Consolidation_Path = Path+'Consolidation\\'


dfs = []

for file_name in os.listdir(Consolidation_Path):
    if file_name.endswith('.csv'):
        
        print(file_name)
        
        df = pd.read_csv(os.path.join(Consolidation_Path, file_name))
        dfs.append(df)

Consolidation = pd.concat(dfs, ignore_index=True)

#Consolidation.to_csv(Consolidation_Path+'Consolidate.csv',index=False)

ATM_V1.csv
ATM_V2.csv


C:\Users\HT185078\AppData\Local\Temp\3\ipykernel_29436\150145941.py:11: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(Consolidation_Path, file_name))


ATM_V3.csv
ATM_V4.csv
ATM_V5.csv
ATM_V6.csv
ATM_V7.csv
ATM_V8.csv
ATM_V9.csv


In [11]:
Consolidation = Consolidation[['Org Code', 'Region', 'Customer_End Customer','Booked Dt','PID','PO No.','filref']]

Consolidation.drop_duplicates(subset=['PID','PO No.', 'Booked Dt'], inplace=True)

Consolidation = Consolidation[Consolidation['PO No.'].notnull()]

Consolidation = Consolidation[Consolidation['Booked Dt'].notnull()]

Consolidation['PO No.'] = Consolidation['PO No.'].astype(str)

val = {'PO No.':'PO','PID':'PN'}
Consolidation.rename(columns= val, inplace=True)
Consolidation['PO'] = Consolidation['PO'].astype(str)

Consolidation['PO'] = Consolidation['PO'].str.replace('.0', '', regex=False)
Data['PO'] = Data['PO'].str.replace('.0', '', regex=False)

In [12]:
first_dt = Consolidation.copy()
last_dt = Consolidation.copy()

first_dt.sort_values(by=['filref'])
first_dt.drop_duplicates(subset=['PN','PO'], keep='first', inplace=True)

last_dt.sort_values(by=['filref'])
last_dt.drop_duplicates(subset=['PN','PO'], keep='last', inplace=True)

In [13]:
Data = pd.merge(Data , first_dt, on=['PN','PO'], how='left')
Data = pd.merge(Data , last_dt, on=['PN','PO'], how='left')

In [14]:
Data['Booked Dt'] = Data['Booked Dt_y']
Data.rename(columns={'Region_x':'Region'}, inplace=True)
Data['Customer Name'] = Data['Customer_End Customer_x']
Data['Country'] = Data['Org Code_x']


In [15]:
# Convert each column to datetime format
date_columns = [
    'SHIPPED DATE',
    'Booked Dt',
    'Promise Dt (first)',
    'Promise Dt (last)',
    'Need By dt'
]

for col in date_columns:
    Data[col] = pd.to_datetime(Data[col])


### Create Dummy Columns

In [16]:
val= ['Act Lt','Act Lt Wks','Act LT Wks Oride','System LT','System LT Wks','Act LT vs System LT','Factory LT','Factory LT Wks','Inventory Item Serial Number','Tracer Number','Manufactured Dt','SSD Wk','Month','Detail comments','Responsibility','Reason','Reason Code','Dataref','Reason Code Groom']

for col in val:
    Data[col] = None


In [17]:
Data[['site','PO','Prod Grp','Class','PN','PID Type','SHIPPED DATE','Ship Period','Qty','Need By dt','Booked Dt','Promise Dt (first)','Promise Dt (last)','Receipt Dt','PO Creation Dt','Delivery vs Orig PD','Complianc Orig PD','Perf Category Orig PD','Act Lt','Act Lt Wks','Act LT Wks Oride','System LT','System LT Wks','Act LT vs System LT','Factory LT','Factory LT Wks','Inventory Item Serial Number','Tracer Number','Manufactured Dt','Offer PF','Customer Name','Country','Region','SSD Wk','Month','Detail comments','Responsibility','Reason','Reason Code','Dataref','Reason Code Groom']].to_excel(Path+'Output\\OTD_Metrics_ENN.xlsx', index=False)